<a href="https://colab.research.google.com/github/SankalpJoshi2910/IBM_DS_CAPSTONEPROJECT/blob/main/Segmenting%20And%20Clustering%20Neighborhoods%20in%20the%20city%20of%20Toronto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Capstone Project Notebook** #
### This notebook contains all the working code developed for the completion of IBM DATA SCIENCE Capstone Project  ###

In [49]:
import pandas as pd
import numpy as np


In [50]:
print("Hello, Capstone Project Course!")

Hello, Capstone Project Course!


# **Segmenting And Clustering Neighborhoods in the city of Toronto** #
## **Part 1** - Generating the Dataframe ##
### In this part the data will be scraped from the internet and data preparation process will be performed on the same to create the final dataframe.###

In [59]:
!pip install wikipedia
import wikipedia as wp
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', 10)

In [52]:
#Get the html source
html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")
df = pd.read_html(html)[0]
print(df.shape)
df.head()

(180, 3)


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [53]:
#Cleaning the dataframe

# removal of Borough = Not assigend from the df
df1 = df[df['Borough'] != 'Not assigned']

# Combining the neighbourhoods with same Postalcode
df2 = df1.groupby(['Postal Code','Borough'], sort=False).agg(', '.join)
df2.reset_index(inplace=True)
df2.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [60]:
# Replacing the name of the neighbourhoods which are 'Not assigned' with names of Borough
for i in range(0,len(df2["Neighbourhood"])):
  if df2.iloc[i][2] == 'Not assigned':
    df2.iloc[i][2] = df2.iloc[i][1]
  else:
    continue

df2



,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [62]:
#shape of the Dataframe
df2.shape

(103, 3)

###End of Part 1###

# Part 2 - Adding Longitude and Lattitute to Data Table


In [63]:
ll = pd.read_csv('https://cocl.us/Geospatial_data')
ll.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merging both the table###

In [64]:
df3 = pd.merge(df2,ll,on='Postal Code')
df3.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


### End of Part 2 ###

# PART 3 - Exploration and Clusering of Neighbouhood in the city of Toronto

In [65]:
df4 = df3[df3['Borough'].str.contains('Toronto',regex=False)]
df4


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
...,...,...,...,...,...
92,M5W,Downtown Toronto,Stn A PO Boxes,43.646435,-79.374846
96,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675
97,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160


### Visualizing above data frame###


In [67]:
import folium
toronto_map = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

for lat,lng,borough,neighbourhood in zip(df4['Latitude'],df4['Longitude'],df4['Borough'],df4['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(toronto_map)
toronto_map

## K- Means Clustering##

In [68]:
from sklearn.cluster import KMeans
k=5
toron_clust = df4.drop(['Postal Code','Borough','Neighbourhood'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(toron_clust)
kmeans.labels_
df4.insert(0, 'Cluster Labels', kmeans.labels_)

In [70]:
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
map_clusters = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighbourhood, cluster in zip(df4['Latitude'], df4['Longitude'], df4['Neighbourhood'], df4['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### End of Part 3###